It is trivial to represnt a univariate polynomial on a computer simply as a list of coefficients for the terms. However creating a representation for a polynomial with multiple indeterminates is significantly more complicated. In particular we need to keep track of what indeterminates exist with what power in each and every term. Creating this directly would be a huge hassle.

A better way to handle this problem is to create a very restricted computer algebra system that allows us to build up the polynomials we want from simpler parts. I can't take credit for this idea which I got from a 2004 paper by Brent Dingle.

The first step is an Atom which just represents a single indeterminate raised to a power.

In [1]:
class Atom:    
    def __init__(self,s,p=1):
        assert type(s) == str
        assert len(s) == 1
        assert type(p) == int
        assert p >= 1
        
        self.s = s
        self.p = p

We will define operations with Atoms so that specifying *p* isn't necessary in general.

Next we will define a Particle which is a product of some Atoms with some coefficient that isn't an Atom. By not requiring that the coefficient be of any particular type we gain some forward compatibility if we have other numeric types. We also do not require that the Particle contain any Atoms at all which allows a Particle to be a constant.

In [2]:
class Particle:    
    def __init__(self,A,coef=1):
        assert type(A) == list
        assert all([type(a) == Atom for a in A])
        self.A = sorted(A)
        self.coef = coef

Note that the list of Atoms is sorted. We will see how this is done a bit later.

Finally we have the MVPoly class which consists of a sum of several particles represented as a list.

In [3]:
class MVPoly:
    def __init__(self,terms):
        self.terms = poly_merge(sorted(terms))


Again we will introduce what it means to sort and merge the terms of an MVPoly object later. 

To start with we want our classes to display themselves in a way that is easily comprehensible. This will make it much easier to check that things are working properly as we continue. For Atoms it is very simple. We just need to show the Atom being rasied to a power. The only special case is when the power is equal to 1 when we leave it out.

In [ ]:
    def __str__(self):
        """Print the Atom"""
        if self.p == 1:
            return f"{self.s}"            
        return f"{self.s}^{self.p}"

For Particles, which could contain any number of Atoms, along with a coefficient things are singificantly more complicated with a lot of special cases. However having defined the way that Atoms should print simplified things a lot.

In [ ]:
    def __str__(self):
        """Print the Particle"""
        if self.A == []:
            return str(self.coef)
        if self.coef == 1:
            out = ""
        elif self.coef == -1:
            out = "-"
        else:
            out = str(self.coef)
        for a in self.A:
            out += str(a)
        return out

Fortunately getting a polynomial to print correctly is fairly easy.

In [ ]:
    def __str__(self):
        """Print the MVPoly"""
        out = str(self.terms[0])
        for term in self.terms[1:]:
            sgn = "-" if term.coef < 0 else "+"
            out +=  " " + sgn + " " + str(abs(term))
        return out

However getting a polynomial represented correctly involves more than just printing out the terms. We need to make sure that we have the correct terms and that they are ordered in way that makes some sense. Atoms we will just sort in terms of the symbol for their indeterminate. This is what is typical when actually writing polynomials.

In [ ]:
    def __lt__(self,other):
        """For sorting atoms"""
        assert type(other) == Atom
        return self.s < other.s

As we saw above Particles use this when they are created. This ensure that identical Particles that have been given the Atoms in a different order still end up the same.

Ordering Particles is a bit complicated and arguably has no best method. Here we first sort to put Particles with many indeterminates to the left. For Particles with the same number of coefficients the ones with higher powers are sorted to the left.

In [ ]:
    def __lt__(self,other):
        """For sorting Particles"""
        assert type(other) == Particle
        # First sort by number of indeterminates
        if len(self.A) != len(other.A):
            return len(self.A) > len(other.A)
        # If they are equal sort by the coefficients
        x = sum([a.p for a in self.A])
        y = sum([a.p for a in other.A])
        return x > y